# CNN attack using Skywater non-linearized data

The goal of this notebook is to correctly preprocess the given data as tensors, train and test a CNN model.

In [1]:
# dataloader.py
# Necessary imports
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
import re

## Creating dataloaders
Used files from Kareem's GitHub repo. I may have made mistakes in sampling the data, so feel free to change anything that has been configured incorrectly.

### Helper functions - Dataloaders

In [2]:
# dataloader.py
# Returns list of files with given format 
def get_files(directory, format, digital_index=0):

    format = re.compile(format)
    files = os.listdir(directory)

    #file_dict = {}
    file_list = [] # fname, fpath, label

    for fname in files:
        if match := format.match(fname):
            fpath = os.path.join(directory, fname)

            dvalue = int(match.groups()[digital_index])
            
            file_list.append((fname, fpath, dvalue))

            #if dvalue in file_dict:
            #    file_dict[dvalue].append(fpath)
            #else:
            #    file_dict[dvalue] = [fpath]

    return file_list #file_dict, file_path

# Creates dataset with given traces
class TraceDataset(Dataset):
    cached_traces = {}
    trace_list    = []

    def __init__(self, file_list, cache=True):
        self.file_list = file_list
        self.cache     = cache

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        fname, fpath, label = self.file_list[index]
        label = self.process_label(label)

        if self.cache and fname in self.cached_traces:
            return self.cached_traces[fname], label
        else:
            return self.load_trace(fname, fpath), label

    def get_info(self, index):
        return self.file_list[index]

    def load_trace(self, fname, fpath):
        with open(fpath, 'r') as file:
            header = file.readline()
            #time_arr = []
            valu_arr = []
            # Fixed error of float32 incorrectly translating values
            for line in file.readlines():
                time, value = line.strip().split()
                match = re.search(r"(?<=e-)\d+", value)
                if match:
                    strip_val = value[0:8]
                    # instead of manual string manipulation via index, used split for safety
                    if(match.group() != "04"):
                        integer_part, fractional_part = strip_val.split('.')
                        new_integer_part = integer_part + fractional_part[:4 - int(match.group())]
                        new_fractional_part = fractional_part[4 - int(match.group()):] 
                        strip_val = new_integer_part + '.' + new_fractional_part
                #time_arr.append(np.float32(time))
                valu_arr.append(np.float32(strip_val))

        trace = np.array(valu_arr, dtype=np.float32)

        if self.cache: 
            self.cached_traces[fname] = trace
            self.trace_list.append(trace)

        return trace
    
    def process_label(self, label): return label

    def cache_all(self):
        assert self.cache == True

        print("Caching all traces")
        for fname, fpath, label in self.file_list:
            self.load_trace(fname, fpath)
        print("DONE Caching all traces")

class TraceDatasetBW(TraceDataset):
    def __init__(self, file_list, bit_select, cache=True):
        self.bit_mask = 1 << bit_select
        super().__init__(file_list, cache=cache)

    def process_label(self, label):
        return 1 if label & self.bit_mask else 0

class TraceDatasetBuilder:
    def __init__(self, adc_bitwidth=8, cache=True):
        self.file_list        = []
        self.cache = cache
        self.adc_bits = adc_bitwidth

        self.dataset = None
        self.dataloader = None
        self.datasets = []
        self.dataloaders = []

    def add_files(self, directory, format, label_group):
        ''' Builds list of powertrace files
        Inputs:
            directory   : folder to search for files
            format      : regular expression to match filenames
            label_index : group index for digital output label corresponding to trace
        Outputs:
            list        : [(file_name, file_path, label) ... ]
        '''
        format = re.compile(format)
        fnames = os.listdir(directory)

        for fname in fnames:
            if match := format.match(fname):
                fpath = os.path.join(directory, fname)
                dvalue = int(match.groups()[label_group])

                self.file_list.append((fname, fpath, dvalue))

    def build(self):
        self.dataset = TraceDataset(self.file_list, cache=self.cache)
        for b in range(self.adc_bits):
            self.datasets.append(TraceDatasetBW(self.file_list, b, cache=self.cache))

        if self.cache:
            self.dataset.cache_all()

    def build_dataloaders(self, **kwargs): # batch_size=256, shuffle=True
        self.dataloader = DataLoader(self.dataset, **kwargs)
        self.dataloaders = [DataLoader(dataset, **kwargs) for dataset in self.datasets]

## Creating dataloaders
This section:
1) Creates dataloaders for both the training and testing datasets.
2) Validates and checks the contents of each dataset.
3) Common variables, such as "input_size" are instantiated.

In [3]:
# Hyperparameters for dataloaders
adc_bits = 8

In [4]:
# Create dataloaders
pwd = os.getcwd()
# print(pwd)
# proj_dir = os.path.dirname(os.path.dirname(pwd))
# print(proj_dir)
# data_dir = os.path.join(pwd, 'analog', 'outfiles')

builder  = TraceDatasetBuilder(adc_bitwidth=adc_bits, cache=True)
builder.add_files(os.path.join(pwd, 'sky_Dec_18_2151'), "sky_d(\\d+)_.*\\.txt", 0)
builder.build()
builder.build_dataloaders(batch_size=256, shuffle=True)

# Initialize dataloaders
dataloaders = builder.dataloaders

try:
    print(f"\nTotal number of datasets in dataloaders: {len(dataloaders)}")
except ModuleNotFoundError:
    print("\tERROR: Dataloader not found. Recheck your implementation and try again.")
    raise
except Exception as e:
    print("\tERROR: Dataloader invalid. Recheck your implementation and try again.")
    raise

if not dataloaders or len(dataloaders) != adc_bits:
    print(f"\tERROR: Dataloader size is incorrect: {len(dataloaders) if dataloaders else 0}. It should be: {adc_bits}.")
    print("\tRecheck your implementation and try again.")
    raise ValueError("Invalid dataloader size")

# Extract input_size and validate initial dataloader
input_size, label_num = None, None
for input, labels in dataloaders[0]:
    input_size = input.size(1)
    label_num = len(labels)
    print(f"Input size: {input_size}")
    print(f"Number of labels: {label_num}\n")

    if input.size(0) != label_num:
        print("\tERROR: Number of traces per input is different from number of labels.")
        print(f"\tNumber of traces per input: {input.size(0)}")
        print(f"\tNumber of labels: {label_num}\n")
        raise ValueError("Mismatch between input size and number of labels")

# Validate all dataloaders
for idx, dataloader in enumerate(dataloaders):
    for input, labels in dataloader:
        if input.size(0) != label_num:
            print(f"\tERROR: Number of traces per input is incorrect in dataloader {idx}.")
            print(f"\tExpected: {label_num}, but got: {input.size(0)}.")
            raise ValueError("Invalid number of traces per input")

        if input.size(1) != input_size:
            print(f"\tERROR: Size of traces per input is incorrect in dataloader {idx}.")
            print(f"\tExpected: {input_size}, but got: {input.size(1)}.")
            raise ValueError("Invalid trace size")

        if len(labels) != label_num:
            print(f"\tERROR: Number of labels is incorrect in dataloader {idx}.")
            print(f"\tExpected: {label_num}, but got: {len(labels)}.")
            raise ValueError("Invalid label count")

    print(f"Validation for dataloader {idx} complete.")

print("\nAll dataloaders are valid. Proceeding to training...")

Caching all traces
DONE Caching all traces

Total number of datasets in dataloaders: 8
Input size: 3001
Number of labels: 256

Validation for dataloader 0 complete.
Validation for dataloader 1 complete.
Validation for dataloader 2 complete.
Validation for dataloader 3 complete.
Validation for dataloader 4 complete.
Validation for dataloader 5 complete.
Validation for dataloader 6 complete.
Validation for dataloader 7 complete.

All dataloaders are valid. Proceeding to training...


### Setup for CNN
Section where the initial imports and variables for our native CNN.

In [5]:
# CNN code
# Necessary imports
import os
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
import numpy as np
import datetime


### Optimizing CUDA version
Section where we desginate the most updated CUDA version for current GPU.

Implementation in progress, but is a lot of unnecessary work; will do if CPU training speeds are unformidable.

## Training CNNs - Native CNN
Using a native CNN, we train each CNN until all of them reaches an accuracy of 1.0.

I'm not sure if aiming for an accuracy of 1.0 is beneficial, as it is just overfitting the model to the training data. A more realistic value may be 0.99, but will set it to 1.0 for the current simulated environment.

The training function automatically reduces the learning rate used in Adam based on target accuracy. Currently testing different values and decrease rates. 

### Hyperparameters

1) def_lr: Default learning rate. Change this value to test different learning rate values.
2) max_epoch: Limit of maximum training epochs; training ends if it reaches this amount of epochs.

In [6]:
# Main hyperparameters
def_lr = 1e-4
max_epoch = 100000

### CNN model initialization
Function that initializes the CNN architecture.

The architecture of the original CNN from the S2ADC paper is as follows:

C = Convolution layer, [filter_num, kernel_size, stride]

MP = Max Pool layer, [pool_zie, stride]

FC = Fully Connected layer, [neuron_num]

OUT = Output layer(another FC), [neuron_num]

C1[5,5,1]

MP1[5,5]

C2[5,5,1]

MP2[5,5]

Flattening layer

FC1[100]

FC2[100]

FC3[100]

OUT[2]

This architecture would most likely be needed to be tweaked for our usecase as our ADC is significiantly more complex.

In [7]:
# Class that defines our CNN model
class psa_CNN(nn.Module):
    def __init__(self):
        super(psa_CNN, self).__init__()
        # 1-D convolution layers
        # Activation = ReLU
        # 8 filters, kernel size = 5, stride = 1, zero padding
        # input = [label_num, input_size, 1]
        # output = [label_num, output_size, (input_size + 2p - d(k-1)-1)/s + 1]
        # = [label_num, output_size, (input_size + k)/s + 1]
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=8, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv1d(in_channels=8, out_channels=8, kernel_size=5, stride=1, padding=2)
        self.conv3 = nn.Conv1d(in_channels=8, out_channels=8, kernel_size=5, stride=1, padding=2)
        self.conv4 = nn.Conv1d(in_channels=8, out_channels=8, kernel_size=5, stride=1, padding=2)
        self.conv5 = nn.Conv1d(in_channels=8, out_channels=8, kernel_size=5, stride=1, padding=2)
        self.conv6 = nn.Conv1d(in_channels=8, out_channels=8, kernel_size=5, stride=1, padding=2)
        # Pooling layers
        # MaxPool, pooling size = 3, stride = 3
        # input = [label_num, input_size, 1]
        # output = [label_num, (input_size + 2p - d(k-1) - 1)/s + 1, 1]
        # = [label_num, (input_size - 3)//3 + 1, 1]
        self.pool1 = nn.MaxPool1d(kernel_size=3, stride=3)
        self.pool2 = nn.MaxPool1d(kernel_size=3, stride=3)
        self.pool3 = nn.MaxPool1d(kernel_size=3, stride=3)
        self.pool4 = nn.MaxPool1d(kernel_size=3, stride=3)
        self.pool5 = nn.MaxPool1d(kernel_size=3, stride=3)
        self.pool6 = nn.MaxPool1d(kernel_size=3, stride=3)
        # Fully connected layers
        # Activation = ReLU
        # 100 neurons
        self.fc1 = nn.Linear(96, 500)
        self.fc2 = nn.Linear(500, 500)
        self.fc3 = nn.Linear(500, 500)
        # Output layer
        # 2 neurons, softmax
        self.o1 = nn.Linear(500, 2)

    def forward(self, x):
        x = x.unsqueeze(1)
        # First convolutional layer
        x = torch.relu(self.conv1(x))
        x = self.pool1(x)
        x = torch.relu(self.conv2(x))
        x = self.pool2(x)
        x = torch.relu(self.conv3(x))
        x = self.pool3(x)
        x = torch.relu(self.conv4(x))
        x = self.pool4(x)
        x = torch.relu(self.conv5(x))
        x = self.pool5(x)
        '''
        x = torch.relu(self.conv6(x))
        x = self.pool6(x)
        '''
        # Flatten the output
        x = x.view(x.size(0), -1)
        # Fully connected layers
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        # Output layer, softmax
        x = torch.softmax(self.o1(x), dim=1)
        return x

### Helper Functions

In [8]:
''' 
Function that initializes the CNN and its core components.
Inputs:
    None
Returns:
    1) cnn: model used(ResNet18)
    2) criterion: loss function, current default=CrossEntropyLoss
    3) learning_rate: learning rate, current default=def_lr
    4) optimizer: optimizer, current default=Adam
'''
def cnn_init():
    # Model: custom CNN, inspired by the S2ADC paper
    cnn = psa_CNN()
    # Loss function: not specified in paper, using Cross Entropy Loss
    criterion = nn.CrossEntropyLoss()

    # Learning rate: default set to def_lr, adjust accordingly
    # Optimizer: not specified in paper, using Adam
    learning_rate = def_lr
    optimizer = optim.Adam(cnn.parameters(), lr=learning_rate)
    return cnn, criterion, learning_rate, optimizer

''' 
Function that initializes parameters used in training.
Inputs:
    None
Returns:
    TRAINING PARAMETERS
    1) num_epochs: number of maximum epochs per training
    2) max_grad_norm: gradient clipping threshold
'''
def param_init():
    # training parameters
    num_epochs = max_epoch
    max_grad_norm = 1.0

    return num_epochs, max_grad_norm

''' 
Function that loads variables from saved .pth file. Checks if the file is valid.
This function simply receives the checkpoint file and checks if it is valid.
Inputs:
    1) checkpoint_path: string of .pth file name 
    2) device: selected device to run training
Returns:
    1) pth_variables: dictionary that contains received .pth variables.
    List of variables are saved in 'load_values' array.
'''
def load_pth_file(checkpoint_path, device, load_values):
    pth_variables = {}
    # Load .pth file
    try:
        checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=True)
        for values in load_values:
            pth_variables[values] = checkpoint.get(values, 0)
        return pth_variables
    except FileNotFoundError:
        print(f"No checkpoint found at {checkpoint_path}. Starting from scratch.")
        raise
    except KeyError as e:
        missing_keys = {
            'cnn_state_dict': "CNN state dictionary",
            'optimizer_state_dict': "optimizer state dictionary",
            'epoch': "epoch",
            'reached_acc': "recorded accuracy of current epoch",
            'osc_count': "oscillation count"
        }
        key = e.args[0]
        if key in missing_keys:
            if key in ['cnn_state_dict', 'optimizer_state_dict', 'epoch', 'reached_acc']:
                print(f"Critical error: Missing \"{key}\" that stores \"{missing_keys[key]}\". Checkpoint file is invalid.")
                raise
            else:
                print(f"Warning: Missing \"{key}\" that stores \"{missing_keys[key]}\". Default values will be used.")
                if key == 'osc_count':
                    osc_count = 0

        print(f"Unknown parameter \"{key}\" in checkpoint. Recheck file and try again.")
        raise

In [9]:
cnns = []
dataloaders = builder.dataloaders 
timestamp   = datetime.datetime.now().strftime('%Y%m%d_%H%M')

print(f"Installed CUDA version: {torch.version.cuda}\n")

# Create CNN per dataset
for i in range(7,-1,-1):
    print(f"Starting training for \"cnn_{i}\"...")

    # initialize CNN
    cnn, criterion, learning_rate, optimizer = cnn_init()
    # Save default learning rate
    default_lr = learning_rate
    # initialize parameters
    num_epochs, max_grad_norm = param_init()

    # Append CNN to cnns array
    cnns.append(cnn)

    # Create checkpoint to save progress
    checkpoint_path = f"cnn_checkpoint_{i}.pth"
    
    # Set device to cuda if available
    if torch.cuda.is_available():
        print("\tGPU found, running training on GPU...")
        device = torch.device("cuda")
        cnn = cnn.to(device)
    else:
        print("\tNo GPU found, running training on CPU...")
        print("\tRecheck CUDA version and if your GPU supports it.")
        device = torch.device("cpu")

    max_acc = 0
    max_acc_epoch = 0

    # Attempt to load saved .pth file
    # start_epoch = LOCAL variable that specifies starting epoch number
    # prev_acc = LOCAL variable that specifies accuracy achieved by loaded .pth file
    load_values = ['cnn_state_dict', 'optimizer_state_dict', 'epoch', 'reached_acc']
    try:
        pth_vars = load_pth_file(checkpoint_path, device, load_values)
    except FileNotFoundError:
        start_epoch = 0
        prev_acc = 0
        osc_count = 0
    else:
        # Manually create variables per value
        cnn.load_state_dict(pth_vars['cnn_state_dict'])
        optimizer.load_state_dict(pth_vars['optimizer_state_dict'])
        start_epoch = pth_vars['epoch']
        prev_acc = pth_vars['reached_acc']
        max_acc = prev_acc
        max_acc_epoch = start_epoch
    
        print(f"Checkpoint loaded. Resuming from epoch {start_epoch}.")
        print(f"\tPrevious reached accuracy: {prev_acc}.")

    # Skip training if target accuracy is reached
    if prev_acc == 1:
        print(f"\tSkipping training: accuracy of 1 already achieved.\n")
        continue

    # Start training
    for e in range(start_epoch, num_epochs):
        try:
            correct = 0
            cnn.train()

            for inputs, labels in dataloaders[i]:
                inputs, labels = inputs.to(device), labels.to(device)
                # Add dimensions for channels and width
                # inputs = inputs.unsqueeze(1).unsqueeze(-1)
                optimizer.zero_grad()
                output = cnn(inputs)
                # Check for NaN in outputs
                if torch.isnan(output).any():
                    print("NaN detected in cnn outputs.")
                    break

                loss = criterion(output, labels)
                # Check for NaN in loss
                if torch.isnan(loss):
                    print("NaN detected in loss. Stopping training.")
                    break
                # print(f"Loss: {loss.item()}")
                loss.backward()

                # Gradient clipping
                torch.nn.utils.clip_grad_norm_(cnn.parameters(), max_grad_norm)
                optimizer.step()
                _, predicted = torch.max(output, 1)
                correct += (predicted == labels).sum()
            
            accuracy = correct / 256
            if accuracy > max_acc:
                max_acc = accuracy
                max_acc_epoch = e + 1

            # Change rate of update for printing accuracy accordingly
            if (e + 1) % 50 == 0:
                print(f'\tTRAINING: cnn[{i}], Epoch {e+1}, Loss: {loss.item()}')
                print(f'\tTRAINING: cnn[{i}], Epoch {e+1}, Accuracy: {accuracy}')
                print(f'\tMax accuracy: {max_acc}')
                print(f'\tMax accuracy epoch: {max_acc_epoch}')

                # Save checkpoint
                torch.save({
                    'cnn_state_dict': cnn.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'epoch': e + 1,
                    'reached_acc': accuracy
                }, checkpoint_path)
                print(f"Checkpoint saved for epoch {e + 1}")

            prev_acc = accuracy

            if accuracy == 1:
                print(f"Reached accuracy of 1. Stopping training for \"cnn_{i}\".")
                print(f"Achieved epoch: {max_acc_epoch}\n")
                # Save checkpoint
                torch.save({
                    'cnn_state_dict': cnn.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'epoch': e + 1,
                    'reached_acc': accuracy
                }, checkpoint_path)
                break
        except KeyboardInterrupt:
            # Save checkpoint
            torch.save({
                'cnn_state_dict': cnn.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'epoch': e + 1,
                'reached_acc': accuracy
            }, checkpoint_path)
            print(f"Training interrupted. Pausing training...")
            print(f"Checkpoint saved for last epoch {e + 1}")
            print(f'\tFinal accuracy: {accuracy}')
            raise


Installed CUDA version: None

Starting training for "cnn_7"...
	No GPU found, running training on CPU...
	Recheck CUDA version and if your GPU supports it.
No checkpoint found at cnn_checkpoint_7.pth. Starting from scratch.
	TRAINING: cnn[7], Epoch 50, Loss: 0.6494647264480591
	TRAINING: cnn[7], Epoch 50, Accuracy: 0.98828125
	Max accuracy: 0.98828125
	Max accuracy epoch: 50
Checkpoint saved for epoch 50
Reached accuracy of 1. Stopping training for "cnn_7".
Achieved epoch: 53

Starting training for "cnn_6"...
	No GPU found, running training on CPU...
	Recheck CUDA version and if your GPU supports it.
No checkpoint found at cnn_checkpoint_6.pth. Starting from scratch.
	TRAINING: cnn[6], Epoch 50, Loss: 0.6622064709663391
	TRAINING: cnn[6], Epoch 50, Accuracy: 0.51953125
	Max accuracy: 0.70703125
	Max accuracy epoch: 8
Checkpoint saved for epoch 50
	TRAINING: cnn[6], Epoch 100, Loss: 0.6227927803993225
	TRAINING: cnn[6], Epoch 100, Accuracy: 0.53125
	Max accuracy: 0.70703125
	Max accurac

KeyboardInterrupt: 

### Testing the CNNs

Using the resulting CNNs saved in the .pth files, we test each CNNs using the provided power traces.

Currently the testing data is a subset of the training data. In the future, if we can generate more data, we will be able to use separate datasets.



In [ ]:
# Run evaluation using testing data
# CURRENTLY USING TRAINING DATA TO TEST DATA: use separate dataset in the future
for i in range(7,-1,-1):
    try:
        checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=True)
        cnn.load_state_dict(checkpoint['cnn_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch']
        print(f"Checkpoint loaded. Resuming from epoch {start_epoch}")
        reached_target_acc = checkpoint['reached_acc']
        print(f"Previous reached accuracy: {reached_target_acc}")
        if reached_target_acc != 1:
            print(f"cnn_{i}\" did not reach accuracy of 1, skipping evaluation.\n\n")
            continue
    except FileNotFoundError:
        print(f"No checkpoint found for \"cnn_{i}\". Starting from scratch.")

    cnns[i].eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloaders[i]:
            inputs, labels = inputs.to(device), labels.to(device)
            # Add dimensions for channels and width
            inputs = inputs.unsqueeze(1).unsqueeze(-1)
            outputs = cnns[i](inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Accuracy: {100 * correct / total:.2f}%")